<a href="https://colab.research.google.com/github/MariliaRot/Covid-19/blob/master/COVID_DADOS_DATAFRAME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise exploratoria de dados - COVID19 no Ceará

Objetivo = fazer a predição de casos utilizando o modelo SIR (Sucetiveis - Infectados - Recuperados)



In [7]:
# Montando o Google Drive para obtenção da base de dados
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
#Manipulando dataframe
import openpyxl
import pandas as pd
import numpy as np
import io
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
plt.style.use('seaborn-whitegrid')
from openpyxl import load_workbook
import ssl


#fonte dos dados: https://github.com/integrasus/api-covid-ce
#dicionarios dos dados: https://github.com/integrasus/api-covid-ce


caminho_dados_br = '/content/drive/My Drive/Colab Notebooks/PAINEL_MINISTERIO_DA_SAUDE-BR/HIST_PAINEL_COVIDBR_26ago2020.csv'
caminho_dados_ce = '/content/drive/My Drive/Colab Notebooks/DADOS-COVID19-CEARA-BRASIL/casos_coronavirus_2020_08_27.csv'

dados_covid_bruto= pd.read_csv(caminho_dados_ce)
#dadosBRbruto = pd.read_csv(caminho_dados_br)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,10,14,16,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,37,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
#analise de resultados de exame
dados_covid_bruto.groupby('resultadoFinalExame').describe()

Unnamed: 0                                ... comorbidadeNeoplasias        
                         count           mean            std  ...                   50% 75% max
resultadoFinalExame                                           ...                              
Em Análise             14857.0  331011.876287  114881.571273  ...                   NaN NaN NaN
Inconclusivo             854.0  406168.396956  153269.021758  ...                   NaN NaN NaN
Negativo              383399.0  282248.941027  205263.019108  ...                   NaN NaN NaN
Positivo              228391.0  503719.308979  160849.323075  ...                   NaN NaN NaN

[4 rows x 48 columns]

In [16]:
type(dados_covid_bruto)

pandas.core.frame.DataFrame

In [17]:
## CEARA ##
#extraindo dados necessarios- selecionando colunas especificas para diminuir o processamento de dados

colunasSelecionadasCovid= [ 'codigoPaciente', 'municipioPaciente',  'sexoPaciente',
 'idadePaciente',
 'dataNotificacao',
 'dataResultadoExame',
 'resultadoFinalExame',
 'obitoConfirmado',
 'dataInicioSintomas',
 'dataObito',
 'classificacaoEstadoSivep',
 'dataInternacaoSivep',
 'dataEntradaUtisSvep',
 'dataSaidaUtisSvep',
 'evolucaoCasoSivep',
 'dataEvolucaoCasoSivep',
  'tipoTesteExame',
 'racaCorPaciente',
 'dataNotificacaoObito',
 'cnesNotificacaoEsus',
 'municipioNotificacaoEsus']

dados_tratados_covidce = dados_covid_bruto.filter(items=colunasSelecionadasCovid)

#excluindo linhas que conhtenham NaN (Not a Number)
dados_tratados_covidce = dados_tratados_covidce.dropna()
#apresentando o cabeçalho dos dados
dados_tratados_covidce.head()

,codigoPaciente,municipioPaciente,sexoPaciente,idadePaciente,dataNotificacao,dataResultadoExame,resultadoFinalExame,obitoConfirmado,dataInicioSintomas,dataObito,classificacaoEstadoSivep,dataInternacaoSivep,dataEntradaUtisSvep,dataSaidaUtisSvep,evolucaoCasoSivep,dataEvolucaoCasoSivep,tipoTesteExame,racaCorPaciente,dataNotificacaoObito,cnesNotificacaoEsus,municipioNotificacaoEsus
2692,41406e5933e697ad0c6637f2f1178189,ICO,MASCULINO,53.0,2020-07-02 00:00:00.000,2020-06-28 00:00:00.000,Positivo,True,2020-06-22 00:00:00.000,2020-07-07 00:00:00.000,Positivo,2020-06-27 00:00:00.000,2020-06-27 00:00:00.000,2020-07-07 00:00:00.000,Óbito,2020-07-07 00:00:00.000,RT PCR,Parda,2020-07-08 12:04:55.293,2611309,ICO
3033,6d1642ee604049f958edbf3ee7a2d84d,ICO,FEMININO,75.0,2020-07-30 00:00:00.000,2020-07-28 00:00:00.000,Positivo,True,2020-07-27 00:00:00.000,2020-08-05 00:00:00.000,Positivo,2020-07-27 00:00:00.000,2020-07-30 00:00:00.000,2020-08-05 00:00:00.000,Óbito,2020-08-05 00:00:00.000,RT PCR,Parda,2020-08-05 14:26:53.250,2611309,ICO
3053,f925ebc390eaf1958de8b5d67068a5d3,FORTALEZA,FEMININO,72.0,2020-05-05 00:00:00.000,2020-05-13 13:57:06.000,Positivo,True,2020-05-02 00:00:00.000,2020-05-09 00:00:00.000,Positivo,2020-05-08 00:00:00.000,2020-05-08 00:00:00.000,2020-05-09 00:00:00.000,Óbito,2020-05-09 00:00:00.000,RT PCR,Parda,2020-05-20 17:35:15.347,704802013610947,FORTALEZA
3085,6b065a00d1e82690dd1d4660e969e833,FORTALEZA,FEMININO,81.0,2020-06-03 00:00:00.000,2020-06-03 00:00:00.000,Positivo,True,2020-05-30 00:00:00.000,2020-07-04 00:00:00.000,Positivo,2020-06-03 00:00:00.000,2020-06-11 00:00:00.000,2020-07-04 00:00:00.000,Óbito,2020-07-04 00:00:00.000,RT PCR,Parda,2020-07-08 13:21:17.337,2529068,FORTALEZA
6498,22a42cfcff30733432ed40a20546cb10,FORTALEZA,MASCULINO,84.0,2020-06-03 00:00:00.000,2020-06-07 15:52:56.000,Positivo,True,2020-06-01 00:00:00.000,2020-06-10 00:00:00.000,Positivo,2020-06-03 00:00:00.000,2020-06-08 00:00:00.000,2020-06-10 00:00:00.000,Óbito,2020-06-10 00:00:00.000,RT PCR,Parda,2020-06-17 13:11:27.887,7429398,FORTALEZA


In [30]:
#identificando a indade minima e max de idade e a media de infectados.
dados_tratados_covidce['idadePaciente'].describe()

count    199.000000
mean      68.874372
std       14.072658
min       23.000000
25%       61.000000
50%       71.000000
75%       79.000000
max       96.000000
Name: idadePaciente, dtype: float64

In [31]:
dados_tratados_covidce.groupby('resultadoFinalExame').describe()

idadePaciente                        ...                  
                            count       mean        std  ...   50%   75%   max
resultadoFinalExame                                      ...                  
Positivo                    199.0  68.874372  14.072658  ...  71.0  79.0  96.0

[1 rows x 8 columns]

In [ ]:
#selecionando os dados confirmados, no modelo SIR, esta seleção correponde ao compatimento I
casos_confirmados= dados_tratados_covidce.loc[dados_tratados_covidce["resultadoFinalExame"]=="Positivo",:]

In [ ]:
#apresentando Casos confirmados (I)
casos_confirmados

In [ ]:
datasInfectados = casos_confirmados["dataResultadoExame"]
datas = datasInfectados.sort_values()
datas.value_counts()

In [ ]:
#selecionando os obitos por covid
obitos_covid= casos_confirmados.loc[casos_confirmados["obitoConfirmado"]==True,:]
datasMortes = obitos_covid["dataObito"]
datasMortes =datasMortes.sort_values()
datasMortes.head()

In [ ]:
#casos confirmado que foram CURADOS, no modelo SIR estes correspondem a variavel R
#aqui pode-se definir tempo medio da doenca no individuo, atraves das datas de inicio de sintomas e a data de Evolução do caso
curados_covid= casos_confirmados.loc[casos_confirmados["evolucaoCasoSivep"]=="Cura",:]
curados_covid

In [ ]:
#distribuição de casos confirmados por municipio

COLUNAmunicipioPaciente = casos_confirmados["municipioPaciente"]
COLUNAmunicipioPaciente.value_counts()

In [ ]:
#descobrindo a quantidade de curas e obitos

dist_evolucaoCasoSivep = casos_confirmados.loc[casos_confirmados["evolucaoCasoSivep"]]
dist_evolucaoCasoSivep.value_counts().dropna()